In [1]:
import pandas as pd
import numpy as np
import os
import imageio
import cv2
import pydicom
import matplotlib.pyplot as plt
from skimage.io import imread
from PIL import *
from pathlib import Path

In [2]:
#Path to the images
main_path = '/Volumes/Alekei/Ale/UAQ/Tesis/new_osicdb_ready/'

In [3]:
#Create a list with the patient ids folders
patient_folders = os.listdir(main_path)

#Remove the folder created automatically by macOS
#patient_folders.remove('.DS_Store')

#Printing the number of folder ids found
print(len(patient_folders))

for patient_folder in patient_folders:
    if patient_folder.startswith('._'):
        patient_folders.remove(patient_folder)
        
print(len(patient_folders))
patient_folders

60
30


['ID00078637202199415319443',
 'ID00202637202249376026949',
 'ID00173637202238329754031',
 'ID00180637202240177410333',
 'ID00035637202182204917484',
 'ID00317637202283194142136',
 'ID00108637202209619669361',
 'ID00104637202208063407045',
 'ID00042637202184406822975',
 'ID00020637202178344345685',
 'ID00291637202279398396106',
 'ID00393637202302431697467',
 'ID00422637202311677017371',
 'ID00309637202282195513787',
 'ID00221637202258717315571',
 'ID00134637202223873059688',
 'ID00030637202181211009029',
 'ID00388637202301028491611',
 'ID00381637202299644114027',
 'ID00312637202282607344793',
 'ID00426637202313170790466',
 'ID00186637202242472088675',
 'ID00132637202222178761324',
 'ID00233637202260580149633',
 'ID00136637202224951350618',
 'ID00124637202217596410344',
 'ID00376637202297677828573',
 'ID00009637202177434476278',
 'ID00219637202258203123958',
 'ID00371637202296828615743']

In [4]:
#Get the array of every image and add it to a list
path_images = []

for patient in patient_folders:
    if patient.startswith('._'):
        pass
    else:
        temp_path = main_path+patient+'/'
        images = os.listdir(main_path+patient)
        for image in images:
            if image.startswith('._'):
                pass
            else:
                temp_path_image = temp_path + image
                path_images.append(temp_path_image)

print(len(path_images))
print(path_images[0])

14591
/Volumes/Alekei/Ale/UAQ/Tesis/new_osicdb_ready/ID00078637202199415319443/842.png


In [5]:
def load_images(path_images):
    tmp = []
    for path in path_images:
        tmp.append(imread(path))
    return tmp

In [6]:
images = load_images(path_images)

In [7]:
#Print the result of loading the images
print(type(images[0]))
print(images[0].shape)
#images[0]

#Convert from list to numpy array
print(type(images))
images = np.array(images)
print(type(images), images.shape)

<class 'numpy.ndarray'>
(224, 224, 4)
<class 'list'>
<class 'numpy.ndarray'> (14591, 224, 224, 4)


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test = train_test_split(images, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape)

(11672, 224, 224, 4) (2919, 224, 224, 4)


In [11]:
# Load a pre-trained model (ResNet50)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

2024-08-25 13:21:55.812164: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-08-25 13:21:55.812315: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-25 13:21:55.812328: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-25 13:21:55.813389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-25 13:21:55.814152: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
X_train = tf.keras.applications.resnet50.preprocess_input(X_train)
X_test = tf.keras.applications.resnet50.preprocess_input(X_test)

In [13]:
train_features = base_model.predict(X_train, batch_size=8, verbose=1)
test_features = base_model.predict(X_test, batch_size=8)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# Apply K-means clustering
k = 4

kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(train_features)
cluster_labels = kmeans.predict(test_features)